청경채 성장 예측 AI 경진대회

- 개발환경

Window, Local cpu

- 라이브러리 버전

In [9]:
import sys
import pandas as pd
import matplotlib.pyplot as plt

print(f'python version : {sys.version}')
print(f'pandas version : {pd.__version__}')

python version : 3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]
pandas version : 1.4.0


- 데이터 경로

train target : 
../data/open/train_target/CASE_01.csv ...

submission : 
../submission/TEST_01.csv ...
(submission 예시 csv 파일입니다.)


- 훈련 및 추론 코드

submission_code.ipynb

- 코드 간단 설명

1. train_target csv 파일에서 outlier로서 작용할 수 있는 데이터를 scaling했습니다.
2. scaling된 data로 평균값을 낸 후, 이 평균값을 그대로 모든 테스트 케이스에 대해 동일하게 적용했습니다.

In [10]:
import pandas as pd
import os

In [11]:
# warning -> off
pd.set_option('mode.chained_assignment',  None)

# csv file loading
def loadCsv(csv_dir, name):
    dir = os.path.join(csv_dir, name)
    file_names = os.listdir(dir)
    dfs = []
    
    for file_name in file_names:
        df = pd.read_csv(os.path.join(dir, file_name))
        dfs.append(df)    
    return dfs

# scaling loaded data
# scale if rate is over threshold (threshold=2)
def loadPreprocessedTarget(csv_dir, name, split_per_day=True, threshold=2):
    targets = loadCsv(csv_dir, name)
    
    if split_per_day:
        splitted = []
        for target in targets:
            for rate in target['rate']:
                splitted.append(rate)
        targets = splitted
    
    if split_per_day:
        for idx, target in enumerate(targets):
            if target>10:
                targets[idx] /=100
            elif target>threshold:
                targets[idx] /= 10
    else:
        for idx, target in enumerate(targets):
            for i in range(len(target)):
                if target['rate'].iloc[i]>10:
                    target['rate'].iloc[i] /=100
                elif target['rate'].iloc[i]>threshold:
                    target['rate'].iloc[i] /= 10
    return targets

# load scaled target data

csv_dir = "C://Users/rlawo/Desktop/Dacon/brassica_prediction/data/open"
targets = loadPreprocessedTarget(csv_dir, name="train_target", split_per_day=False, threshold=5)

In [12]:
max_len = max([len(target) for target in targets])
total_sum = [0 for i in range(max_len)]
count = [0 for i in range(max_len)]

for target in targets:
    for idx, rate in enumerate(target['rate']):
        total_sum[idx] += rate
        count[idx] += 1

# get mean of all target data
mean = [total_sum[i]/count[i] for i in range(max_len)]

In [13]:
# load submission csv files


csv_dir = "C://Users/rlawo/Desktop/Dacon/brassica_prediction/submission"
file_names = os.listdir(csv_dir)

remove_lst = []
for file_name in file_names:
    if file_name[-4:]=='.zip':
        remove_lst.append(file_name)
for r in remove_lst:
    file_names.remove(r)  

submissions = []

for file_name in file_names:
    submission = pd.read_csv(os.path.join(csv_dir, file_name))
    submissions.append(submission)

In [14]:
# replace submission csv 'rate' column to mean list

for n, submission in enumerate(submissions):
    if n==3:
        submission['rate'] = mean[3:len(submission)+3]
    submission['rate'] = mean[:len(submission)]

In [15]:
save_dir = "C://Users/rlawo/Desktop/Dacon/brassica_prediction/submission"
for idx, submission in enumerate(submissions):
    submission.to_csv(os.path.join(save_dir, "TEST_{0:02d}.csv".format(idx+1)), index=False)